In [ ]:
!pip install customtkinter

In [1]:
import customtkinter as ctk
from tkinter import messagebox
from tensorflow.keras.models import load_model
import pickle
import numpy as np
from nltk.tokenize import TweetTokenizer 

# Load model
model = load_model("CONV1D_LSTM_31acc.keras")  

# Load vocab dictionary
vocab = None
with open("vocab.pkl", "rb") as f:
    vocab = pickle.load(f)

emoji_map = {
    0: "❤",
    1: "😍",
    2: "😂",
    3: "💕",
    4: "🔥",
    5: "😊",
    6: "😎",
    7: "✨",
    8: "💙",
    9: "😘",
    10: "📷",
    11: "🇺🇸",
    12: "☀",
    13: "💜",
    14: "😉",
    15: "💯",
    16: "😁",
    17: "🎄",
    18: "📸",
    19: "😜"
}


def tokenizeText(text): 
    tknzr = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True) 
    return tknzr.tokenize(text.lower())

def get_emoji_from_text(text):
    text = text.lower() 
     
    maxLen=40 
    wordVec = np.zeros(maxLen)
    words = tokenizeText(text)
    for j in range(min(len(words),maxLen)):
        if(vocab.get(words[j])) : wordVec[j]=vocab[words[j]]
    x = np.array([wordVec])
    # print(x)

    Y_predicted = model.predict(x)[0] 

    Y = [(Y_predicted[i],i) for i in range(len(Y_predicted))] 
    Y.sort(key=lambda a:-a[0]) 
    top3 = [it[1] for it in Y[0:3]]

    result = "" 
    for y in top3: 
        result += emoji_map[y]
        
    return result

def analyze_text():
    user_text = text_box.get("0.0", "end").strip()
    if not user_text:
        messagebox.showwarning("Empty Input", "Please type some text first!")
        return
    emoji = get_emoji_from_text(user_text)
    emoji_label.configure(text=emoji)

# --- App Setup ---
ctk.set_appearance_mode("dark")        # Modes: "light" or "dark"
ctk.set_default_color_theme("blue")    # Themes: "blue", "green", "dark-blue"

app = ctk.CTk()
app.title("Emotion → Emoji")
app.geometry("420x400")
app.resizable(False, False)

# --- Title ---
title_label = ctk.CTkLabel(
    app, text="💬 Emotion → Emoji 😊",
    font=ctk.CTkFont(size=22, weight="bold")
)
title_label.pack(pady=(25, 10))

# --- Input box ---
text_box = ctk.CTkTextbox(app, width=320, height=100, corner_radius=10)
text_box.pack(pady=10)

# --- Analyze button ---
analyze_button = ctk.CTkButton(
    app, text="Analyze Emotion", 
    command=analyze_text, 
    width=200, height=40, 
    corner_radius=20, font=ctk.CTkFont(size=15, weight="bold")
)
analyze_button.pack(pady=15)

# --- Result label (emoji display) ---
emoji_label = ctk.CTkLabel(
    app, text="✨", font=ctk.CTkFont(size=50, weight="bold")
)
emoji_label.pack(pady=20)

# --- Footer ---
footer_label = ctk.CTkLabel(
    app, text="Type how you feel above 💭", 
    font=ctk.CTkFont(size=12, slant="italic")
)
footer_label.pack(side="bottom", pady=10)

# --- Run app ---
app.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step


In [12]:
test_data = [
    {"text": "I love you so much!", "expected": ["❤", "💕", "😘"]},
    {"text": "This sunset is absolutely gorgeous", "expected": ["☀", "😍", "✨"]},
    {"text": "Hahaha you’re so funny 😂", "expected": ["😂", "😁", "😜"]},
    {"text": "Best friends forever", "expected": ["💕", "❤", "😊"]},
    {"text": "This song is fire", "expected": ["🔥", "❤", "💯"]},
    {"text": "Feeling blessed and happy today", "expected": ["😊", "❤", "✨"]},
    {"text": "Just chilling with my shades on", "expected": ["😎", "🔥", "😉"]},
    {"text": "Everything is shining so bright", "expected": ["✨", "☀", "😊"]},
    {"text": "Blue skies and good vibes", "expected": ["💙", "☀", "✨"]},
    {"text": "Sending kisses your way", "expected": ["😘", "💕", "❤"]},
    {"text": "Took this amazing photo today", "expected": ["📷", "📸", "✨"]},
    {"text": "Proud to be an American", "expected": ["🇺🇸", "💯", "❤"]},
    {"text": "Such a beautiful sunny day", "expected": ["☀", "😊", "✨"]},
    {"text": "Purple is my favorite color", "expected": ["💜", "💙", "✨"]},
    {"text": "Winking at my crush", "expected": ["😉", "😘", "😍"]},
    {"text": "You did an amazing job!", "expected": ["💯", "🔥", "😊"]},
    {"text": "Feeling great today!", "expected": ["😁", "😊", "✨"]},
    {"text": "Can’t wait for Christmas!", "expected": ["🎄", "😊", "❤"]},
    {"text": "Captured the perfect shot", "expected": ["📸", "📷", "🔥"]},
    {"text": "You’re such a tease haha", "expected": ["😜", "😉", "😂"]}, 
    {
        "text": "Just got back from the beach and watched the sunset with my best friends. The sky was glowing orange and pink — absolutely perfect.",
        "expected": ["☀", "✨", "💕"]
    },
    {
        "text": "After months of hard work, I finally got promoted today! Feeling so proud and thankful for everyone who supported me.",
        "expected": ["💯", "🔥", "😊"]
    },
    {
        "text": "Spent the evening cooking dinner for my partner, listening to soft music, and laughing the whole time. Love nights like this.",
        "expected": ["❤", "💕", "😘"]
    },
    {
        "text": "Just posted some new travel photos from our trip to Italy — the views were unreal and the food was amazing!",
        "expected": ["📸", "📷", "✨"]
    },
    {
        "text": "It’s snowing outside and the lights on the tree are sparkling. Christmas vibes all around the house tonight.",
        "expected": ["🎄", "✨", "😊"]
    },
    {
        "text": "Feeling down today but trying to stay positive and remind myself of all the good things I have in life.",
        "expected": ["😊", "❤", "💙"]
    },
    {
        "text": "Had the best weekend ever — road trip with the crew, loud music, crazy laughs, and memories for life!",
        "expected": ["😎", "😂", "🔥"]
    },
    {
        "text": "The morning sunshine hit my window and I instantly felt happier. Ready to take on the day with good energy!",
        "expected": ["☀", "😊", "✨"]
    },
    {
        "text": "Can’t stop laughing at the memes my brother keeps sending me. My stomach hurts from laughing so much!",
        "expected": ["😂", "😁", "😜"]
    },
    {
        "text": "Wore my favorite purple hoodie today and got so many compliments. Feeling cute and confident.",
        "expected": ["💜", "😊", "😉"]
    }
]


In [ ]:
# Load model
model = load_model("CONV1D_LSTM_31acc.keras")  

# Load vocab dictionary
vocab = None
with open("vocab.pkl", "rb") as f:
    vocab = pickle.load(f) 

def get_emoji_from_text(text):
    text = text.lower() 
     
    maxLen=40 
    wordVec = np.zeros(maxLen)
    words = tokenizeText(text)
    for j in range(min(len(words),maxLen)):
        if(vocab.get(words[j])) : wordVec[j]=vocab[words[j]]
    x = np.array([wordVec])
    # print(x)

    Y_predicted = model.predict(x)[0] 

    Y = [(Y_predicted[i],i) for i in range(len(Y_predicted))] 
    Y.sort(key=lambda a:-a[0]) 
    top3 = [it[1] for it in Y[0:3]]

    result = "" 
    for y in top3: 
        result += emoji_map[y]
        
    return result



In [13]:
with open("Results.md","w",encoding="utf-8") as f:
    for t in test_data: 
        f.write("Text:"+t["text"]+"\n")
        f.write("Predicted:"+get_emoji_from_text(t["text"])+"\n") 
        f.write("Expected:"+t["expected"][0]+t["expected"][1]+t["expected"][2]+"\n") 
        f.write("\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━